# Customer Segmentation using RFM Analysis

# Loading Dataset & Exploratory Data Analysis (EDA)

In [1]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np

# Step 2: Load your dataset
df = pd.read_csv("online_retail.csv", encoding='ISO-8859-1')  # use this encoding if there are special characters

# Step 3: Quick look at data
print(df.head())
print(df.info())
print(df.describe())

   index InvoiceNo StockCode                          Description  Quantity  \
0      0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1      1    536365     71053                  WHITE METAL LANTERN         6   
2      2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3      3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4      4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       

# Data Cleaning & Preprocessing

In [2]:
# Drop rows with missing CustomerID
df = df[df['CustomerID'].notnull()]

# Remove negative or zero quantity
df = df[df['Quantity'] > 0]

# Remove duplicates
df = df.drop_duplicates()

# Feature Engineering

In [3]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Date & Time Processing

In [4]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# RFM Metrics Calculation

In [5]:
# Set a reference date (usually the day after last transaction)
import datetime
NOW = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Aggregate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (NOW - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                        # Frequency
    'TotalPrice': 'sum'                            # Monetary
})

# Rename columns
rfm.rename(columns={'InvoiceDate': 'Recency', 
                    'InvoiceNo': 'Frequency', 
                    'TotalPrice': 'Monetary'}, inplace=True)

print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0         326          1  77183.60
12347.0           2          7   4310.00
12348.0          75          4   1797.24
12349.0          19          1   1757.55
12350.0         310          1    334.40


# RFM Scoring (Quantile-Based Segmentation)

In [6]:
# Recency: lower is better → use qcut reversed
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])

# Frequency: higher is better → use qcut
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])

# Monetary: higher is better → use qcut
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

# Combine RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

print(rfm.head())

            Recency  Frequency  Monetary R_Score F_Score M_Score RFM_Score
CustomerID                                                                
12346.0         326          1  77183.60       1       1       5       115
12347.0           2          7   4310.00       5       5       5       555
12348.0          75          4   1797.24       2       4       4       244
12349.0          19          1   1757.55       4       1       4       414
12350.0         310          1    334.40       1       1       2       112


In [7]:
rfm['R_Score'] = rfm['R_Score'].astype(int)
rfm['F_Score'] = rfm['F_Score'].astype(int)
rfm['M_Score'] = rfm['M_Score'].astype(int)

# Customer Segmentation

In [8]:
def rfm_segment(row):
    if row['R_Score'] >= 4 and row['F_Score'] >= 4 and row['M_Score'] >= 4:
        return 'Best Customers'
    elif row['R_Score'] >= 4 and row['F_Score'] <= 2:
        return 'New Customers'
    elif row['R_Score'] <= 2 and row['F_Score'] >= 4:
        return 'Loyal Customers'
    else:
        return 'Others'

In [9]:
rfm['Segment'] = rfm.apply(rfm_segment, axis=1)
rfm['Segment'].value_counts()

Others             2781
Best Customers      962
New Customers       320
Loyal Customers     276
Name: Segment, dtype: int64

# Export for SQL & Power BI

In [10]:
rfm.reset_index().to_csv("rfm_customer_segmentation.csv", index=False)